# Lab 4-2: Load Data (22.06.27)

Author: Seungjae Lee (이승재)

<div class="alert alert-warning">
    We use elemental PyTorch to implement linear regression here. However, in most actual applications, abstractions such as <code>nn.Module</code> or <code>nn.Linear</code> are used.
</div>

## Slicing 1D Array

In [1]:
nums = [0, 1, 2, 3, 4]

In [2]:
print(nums)

[0, 1, 2, 3, 4]


index 2에서 4 전까지 가져와라. (앞 포함, 뒤 비포함)

In [3]:
print(nums[2:4])

[2, 3]


index 2부터 다 가져와라.

In [4]:
print(nums[2:])

[2, 3, 4]


index 2 전까지 가져와라. (역시 뒤는 비포함)

In [5]:
print(nums[:2])

[0, 1]


전부 가져와라

In [6]:
print(nums[:])

[0, 1, 2, 3, 4]


마지막 index 전까지 가져와라. (뒤는 비포함!)

In [7]:
print(nums[:-1])

[0, 1, 2, 3]


assign 도 가능!

In [8]:
nums[2:4] = [8, 9]

In [9]:
print(nums)

[0, 1, 8, 9, 4]


## Slicing 2D Array

In [10]:
import numpy as np

In [11]:
b = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

In [12]:
print(b)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [13]:
b[:, 1]

array([ 2,  6, 10])

In [14]:
b[-1]

array([ 9, 10, 11, 12])

In [15]:
b[-1, :]

array([ 9, 10, 11, 12])

In [16]:
b[-1, ...]

array([ 9, 10, 11, 12])

In [17]:
b[0:2, :]

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

## Loading Data from `.csv` file

In [18]:
import numpy as np

In [20]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [23]:
xy = np.loadtxt('/gdrive/My Drive/AI study/DeepLearningZeroToAll-Pytorch/data-01-test-score.csv', delimiter=',', dtype=np.float32)

In [24]:
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [25]:
print(x_data.shape) # x_data shape
print(len(x_data))  # x_data 길이
print(x_data[:5])   # 첫 다섯 개

(25, 3)
25
[[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]]


In [26]:
print(y_data.shape) # y_data shape
print(len(y_data))  # y_data 길이
print(y_data[:5])   # 첫 다섯 개

(25, 1)
25
[[152.]
 [185.]
 [180.]
 [196.]
 [142.]]


## Imports

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [28]:
# For reproducibility
torch.manual_seed(1)

## Low-level Implementation

In [29]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 26811.960938
Epoch    1/20 Cost: 9920.530273
Epoch    2/20 Cost: 3675.298340
Epoch    3/20 Cost: 1366.260498
Epoch    4/20 Cost: 512.542480
Epoch    5/20 Cost: 196.896637
Epoch    6/20 Cost: 80.190987
Epoch    7/20 Cost: 37.038696
Epoch    8/20 Cost: 21.081343
Epoch    9/20 Cost: 15.178760
Epoch   10/20 Cost: 12.993679
Epoch   11/20 Cost: 12.183023
Epoch   12/20 Cost: 11.880535
Epoch   13/20 Cost: 11.765958
Epoch   14/20 Cost: 11.720851
Epoch   15/20 Cost: 11.701438
Epoch   16/20 Cost: 11.691514
Epoch   17/20 Cost: 11.685116
Epoch   18/20 Cost: 11.680005
Epoch   19/20 Cost: 11.675380
Epoch   20/20 Cost: 11.670952


## High-level Implementation with `nn.Module`

In [30]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [31]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 28693.490234
Epoch    1/20 Cost: 10618.750000
Epoch    2/20 Cost: 3936.015381
Epoch    3/20 Cost: 1465.219727
Epoch    4/20 Cost: 551.693726
Epoch    5/20 Cost: 213.934616
Epoch    6/20 Cost: 89.052269
Epoch    7/20 Cost: 42.876007
Epoch    8/20 Cost: 25.799639
Epoch    9/20 Cost: 19.482420
Epoch   10/20 Cost: 17.143093
Epoch   11/20 Cost: 16.274508
Epoch   12/20 Cost: 15.949721
Epoch   13/20 Cost: 15.825986
Epoch   14/20 Cost: 15.776565
Epoch   15/20 Cost: 15.754660
Epoch   16/20 Cost: 15.742919
Epoch   17/20 Cost: 15.734917
Epoch   18/20 Cost: 15.728307
Epoch   19/20 Cost: 15.722219
Epoch   20/20 Cost: 15.716357


## Dataset and DataLoader

<div class="alert alert-warning">
    pandas 기초지식이 필요할 것 같다
</div>

너무 데이터가 크면 `x_data`, `y_data` 를 전부 다 가져오지 말고, 필요한 배치만 가져올 수 밖에 없다.

[PyTorch Data Loading and Processing tutorial](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#iterating-through-the-dataset)

## Pytorch Dataset

In [32]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
  def __init__(self):
    self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
    self.y_data = [[152], [185], [180], [196], [142]]
  
  def __len__(self):
    return len(self.x_data)
  
  def __getitem__(self, idx):
    x = torch.FloatTensor(self.x_data[idx])
    y = torch.FloatTensor(self.y_data[idx])
    return x, y

dataset = CustomDataset()

## Pytorch DataLoader

In [34]:
from torch.utils.data import DataLoader

dataloader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
)

## Full Code with Dataset and DataLoader

In [35]:
nb_epochs = 20

for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    x_train, y_train = samples
    # H(x) 계산
    prediction = model(x_train)

    # Cost 계산
    cost = F.mse_loss(prediction, y_train)

    # Cost 로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()
    ))

Epoch    0/20 Batch 1/3 Cost: 0.894915
Epoch    0/20 Batch 2/3 Cost: 2.855492
Epoch    0/20 Batch 3/3 Cost: 0.069942
Epoch    1/20 Batch 1/3 Cost: 0.410041
Epoch    1/20 Batch 2/3 Cost: 0.278853
Epoch    1/20 Batch 3/3 Cost: 0.010973
Epoch    2/20 Batch 1/3 Cost: 0.153153
Epoch    2/20 Batch 2/3 Cost: 0.493676
Epoch    2/20 Batch 3/3 Cost: 0.004545
Epoch    3/20 Batch 1/3 Cost: 0.424114
Epoch    3/20 Batch 2/3 Cost: 0.014969
Epoch    3/20 Batch 3/3 Cost: 0.337642
Epoch    4/20 Batch 1/3 Cost: 0.547283
Epoch    4/20 Batch 2/3 Cost: 0.210737
Epoch    4/20 Batch 3/3 Cost: 0.089755
Epoch    5/20 Batch 1/3 Cost: 0.100483
Epoch    5/20 Batch 2/3 Cost: 0.529547
Epoch    5/20 Batch 3/3 Cost: 0.009955
Epoch    6/20 Batch 1/3 Cost: 0.227869
Epoch    6/20 Batch 2/3 Cost: 0.044879
Epoch    6/20 Batch 3/3 Cost: 0.929246
Epoch    7/20 Batch 1/3 Cost: 0.217348
Epoch    7/20 Batch 2/3 Cost: 0.659224
Epoch    7/20 Batch 3/3 Cost: 0.002124
Epoch    8/20 Batch 1/3 Cost: 0.465546
Epoch    8/20 Batch 2/3 C